In [1]:
import os
from zipfile import ZipFile
from google.colab import files
uploads=files.upload()

Saving CSE508_Winter2023_Dataset3_preprocessed.zip to CSE508_Winter2023_Dataset3_preprocessed.zip


In [2]:
class Inverted_Index:
  def __init__(self):
    self.index_dict = dict()        # dictionary for storing unigram inverted index
    self.token_list = list()        # list to store all of the words present in the documents
    self.doc_count = 0              # variable to store the total count of documents
    self.word_freq = dict()         # dictionary for storing the count of indices stored for each word in the unigram inverted index
    self.all_doc_ids = []           # list to store all document id numbers
    self.doc_id_map = dict()        # dictionary that maps document_id to document_name


  def make_tok_list(self):
    os.mkdir('temp_dir')
    with ZipFile("/content/CSE508_Winter2023_Dataset3_preprocessed.zip", 'r') as zObject:    
      zObject.extractall(path="temp_dir")                                        # Extracting all the files from the zip into a specific location
    dir_path='/content/temp_dir/content/temp_dir/CSE508_Winter2023_Dataset'
    files_list=os.listdir(dir_path)                                              # getting the files in a list          
    i = 0
    for fl in files_list:
      self.doc_id_map[i] = fl
      self.all_doc_ids.append(i)                        # adding the current file's document_id to the list all_doc_ids
      i += 1
      self.doc_count += 1
      fl_path=os.path.join(dir_path,fl)
      fobj=open(fl_path,'r')
      text=fobj.read()
      word_toks = text.split(",")                       # the words in the file are seperated by comma, to extract them spliting wrt ","
      self.token_list.append(word_toks)                 # then adding those words in the token_list, which is collection of all present tokens
      fobj.close()


  def make_unigram(self):
    unique_token_list=set()                                       # stores unique tokens present in token_list
    self.make_tok_list()
    for toks in self.token_list:
      unique_token_list = unique_token_list.union(set(toks))      # getting the unique token from tokens present in token_list using set union
    unique_token_list = list(unique_token_list)                   
    for tok in unique_token_list:
      self.index_dict[tok] = []                                   # initializing each inverted index as empty list
      self.word_freq[tok] = 0                                     # initializing the length of each inverted index as zero
    for doc in range(self.doc_count):                             # generating the unigram inverted index
      for tok in self.token_list[doc]:
        if doc not in self.index_dict[tok]:
          self.index_dict[tok].append(doc)
          self.word_freq[tok] = self.word_freq[tok] + 1
    for tok in unique_token_list:
      self.index_dict[tok].sort()                                 # sorting the document_ids present in each inverted index


  def t1_AND_t2(self,doc_list1,doc_list2,l1,l2):         # takes the inverted index list of two operand tokens and their lengths as input
    if l1 == 0 and l2 ==0:                               # if none of the operands present in the index_dict the return empty list
      return [],0
    i, j, comparisons = 0, 0, 0                          # the variable comparisons stores the number of comparisons made for this task
    docs = []
    while i<l1 and j<l2:                                      
      comparisons += 1
      if doc_list1[i] == doc_list2[j]:
        docs.append(doc_list1[i])
        i += 1
        j += 1
      elif doc_list1[i] < doc_list2[j]:
        i += 1
      else:
        j += 1
    return docs,comparisons                              # return the document_ids present in both doc_list1 and doc_list2, and comparisons_count


  def t1_OR_t2(self,doc_list1,doc_list2,l1,l2):          # takes the inverted index list of two operand tokens and their lengths as input
    if l1!=0 and l2!=0:                                  # condition to check if both of the tokens are present in the inverted index
      i, j, comparisons = 0, 0, 0                        # the variable comparisons stores the number of comparisons made for this task
      docs = []
      while i<l1 and j<l2:                           # generating the union of doc_list1 and doc_list2 considering they may have some common items
        comparisons+=1
        if doc_list1[i]<doc_list2[j]:
          docs.append(doc_list1[i])
          i=i+1
        elif doc_list1[i]>doc_list2[j]:
          docs.append(doc_list2[j])
          j=j+1
        else:
          docs.append(doc_list1[i])
          i=i+1
          j=j+1
      docs = docs + doc_list1[i:] + doc_list2[j:]        
      return docs,comparisons                             # return the document_ids present in either doc_list1 or doc_list2, and comparisons_count
    elif l1!=0:                                           # condition to check if only the first token is present in the inverted index
      return doc_list1,0                                  # simply return the inverted index for first token
    elif l2!=0:                                           # condition to check if only the second token is present in the inverted index
      return doc_list2,0                                  # simply return the inverted index for second token
    else:                                                 # if none of the tokens are present in the dictionary the return empty list
      return [],0
